![diagram](Diagram.PNG)

## Plugin definition in common

### Manifest

In [1]:
# Manifest example
from manifests import BruteForceExploiterManifest
from config_fields import NumericField, ListField

manifest = BruteForceExploiterManifest(title="RDP Exploiter", 
                                       plugin_name="rdp_exploiter",  # Could be automatically created somehow?
                                       description="Exploits RDP connections by brute-forcing. Warning! it's slow.", 
                                       link_to_docs="www.beef.face",
                                       is_safe=True,
                                       # Right now no exploiter needs these. Implementing the ability to add options would require a refactoring of plugin selectors to also trigger the view of these
                                       # options
                                       options={
                                           "scan_timeout": NumericField(description="How much time to wait between each brute-force attempt. Increase to make the exploitation process quieter",
                                                                        title="RDP timeout"),
                                            "rdp_versions": ListField(all_values=["1.*", "2.*", "3.*"],
                                                                      title="Target versions",
                                                                      description="Which versions of RDP this plugin should target")
                                       })


### Config

In [2]:
# Config example

config = {
    "plugin_name": "rdp_exploiter",
    "enabled": True,
    # Don't validate these for now, as we only need the hard-coded smb-timeout
    "options":{
        "scan_timeout": 10,
        "rdp_versions": ["2.*", "3.*"]
    }
}

### Codebase

In [3]:
# Plugin codebase example (notice how it uses the options)
from time import sleep
from common.plugin_registry import register_plugin

def _scan_rdp_version(target):
    pass

def _attempt_brute_force(target, username, password):
    pass

def main(options):
    if not _scan_rdp_version(options['target']['ip']) in options["rdp_versions"]:
        raise RuntimeError(f"Target {options['target']} is running unsupported RDP version")
    for username, password in options['credentials']:
        if _attempt_brute_force(target, username, password):
            # Send exploitation event
            return 
        sleep(options["scan_timeout"])
        
# IDK how we're going to register it, it's just a mock
register_plugin(manifest=manifest, config=config, entrypoint=main)

## Common actions

## Backend

### Loading the plugin

1. Load the plugin (make sure no other plugin with the same name already exists, validate the configuration against manifest)
2. Store configuration in the repository

### Resetting to default

1. Overwrite the configuration stored with the one in plugin definition

### Fetching agent configuration

1. Query an endpoint `/api/configuration/<plugin_name>`

### Setting configuration

1. Validate the configuration against manifest. 
2. Store the configuration in repository. It's probably best to have a repository per config, file name would correspond to the name of the plugin. 

## UI

### Loading configuration page
1. We need to create/mofify the schema based on manifests

### Exporting configuration

No changes

### Importing configuration

Standard procedure, but when it comes to plugins it would be best to import as much as we can. So if we have a config and have imported a new plugin, that config won't go to waste. This is not priority 0 though

### Submitting configuration

Validation is a consideration once we implement plugin options.
There's a list of things we could do for validation. My opinion is that back-end validation is enough. Features:
 - **Backend validation**. Back end validates the options using validators from the manifest. If there are any errors, the errors are displayed in the UI.
 - **Field type validation**. Make sure that field inputs correlate to the manifest, for example number field only accepts numbers, etc.
 - **Regex string**. If there's a regex validator in manifest we could use the same for front-end.

## Island 

### Storing the configuration

Validate the config against manifests

## Requirements

Agent needs to know the current config

Island needs to know default and current config

Interfaces need to know current, all available options and descriptions

### Single config vs separate configurations

Single config:
 - Less traffic
 - Would need to merge plugin configurations into it
 - Configurations are getting coupled
 
Config per plugin:
 - Easier loading
 - Harder resetting and import/export
 - More changes, refactoring repository

# Agent

## Running the plugins

Agent needs to run the entrypoint of the plugin and pass the relevant options. Our config should end up pretty much the same, so doesn't matter much